## Notebook 3: Machine Learning with Mobile Phone Data
Replication code for:
- Figure S10
- Figure S11
- Table S4
- Table S12
- Table S19

In [ ]:
import numpy as np
import sys
import os
import time
import json
import shutil
import dask.dataframe as ddf
import random
from joblib import dump, load
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, roc_auc_score, mean_squared_error
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, Lasso
from sklearn.model_selection import cross_validate, KFold, GridSearchCV, cross_val_predict, cross_val_score
from sklearn.base import BaseEstimator, TransformerMixin, clone
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from lightgbm import LGBMRegressor
from scipy.stats import spearmanr
from joblib import dump, load

from helpers import *

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Cross validation initialization
inner_strat = KFold(n_splits=3, shuffle=True, random_state=15)
outer_strat = KFold(n_splits=5, shuffle=True, random_state=17)

# LGBM Model
lgbm_grid = {'winsorizer__limits':[(0., 1.), (0.005, .995)],
             'model__min_data_in_leaf':[10, 20, 50], 
             'model__num_leaves':[5, 10, 20],
             'model__learning_rate':[0.05, 0.075, 0.1],
             'model__n_estimators':[20, 50, 100]}
lgbm_cv = Pipeline([('winsorizer', Winsorizer()), 
                     ('model', LGBMRegressor(n_jobs=-1, random_state=2))])

In [ ]:
def ml(feats_fname, survey_fname, outcome, toplevel_dir, rural_restriction=False, grid=None):
    
    # Load features
    feats = pd.read_csv(feats_fname)
    print('CDR observations: %i (%i unique)' % (len(feats), len(feats['phone_number'].unique())))
    
    # Load labels
    survey = pd.read_csv(survey_fname)
    if rural_restriction:
        survey = survey[survey['milieu'] == 'rural']
    if outcome in ['pmt', 'consumption', 'cons']:
        survey[outcome] = np.log(survey[outcome])
    print('Survey observations: %i (%i unique)' % (len(survey), len(survey['phone_number'].unique())))
    
    # Merge features and labels
    survey = survey.dropna(subset=[outcome])
    print('Survey observations with outcome: %i (%i unique)' % 
          (len(survey), len(survey['phone_number'].unique())))
    df = survey[['phone_number', outcome, 'weight']].merge(feats, on='phone_number')
    print('Final number of matched observations: %i (%i unique)' % (len(df), len(df['phone_number'].unique())))
    
    # Get X and Y 
    x = df.drop(['phone_number', 'weight', outcome], axis=1)
    y = df[outcome]   
    
    modelname, model, grid = 'LGBM', lgbm_cv, lgbm_grid
    print('Running tuned ' + modelname + '....')
    
    # Make folder structure
    if not os.path.isdir(toplevel_dir):
        os.mkdir(toplevel_dir)
    outfolder = toplevel_dir + modelname + '/'
    if os.path.isdir(outfolder):
        shutil.rmtree(outfolder)
    os.mkdir(outfolder)

    # Train model over all data, tuning hyperparameters with 3 fold CV
    model = GridSearchCV(estimator=model,
                         param_grid=grid,
                         cv=inner_strat,
                         verbose=0,
                         scoring='r2',
                         return_train_score=True,
                         refit=True,
                         n_jobs=-1)
    model.fit(x, y)

    # Write the model itself
    dump(model, outfolder + 'model')

    # Record tuning results
    resultsdf = pd.DataFrame(model.cv_results_)
    resultsdf.to_csv(outfolder + 'tuning.csv', index=False)
    best_row = resultsdf.iloc[resultsdf['mean_test_score'].argmax()]
    print('Best r2 --- Train: %.2f (%.2f), Test: %.2f (%.2f)' % (best_row['mean_train_score'], 
                                                                 best_row['std_train_score'],
                                                                 best_row['mean_test_score'],
                                                                 best_row['std_test_score']))
    print('Best Hyperparameters: ' + json.dumps(resultsdf.iloc[resultsdf['mean_test_score'].argmax()]['params']))

    # Feature importances
    try:
        imports = model.best_estimator_.named_steps['model'].feature_importances_
    except:
        imports = model.best_estimator_.named_steps['model'].coef_
    imports = pd.DataFrame([x.columns, imports]).T
    imports.columns = ['Feature', 'Importance']
    imports = imports.sort_values('Importance', ascending=False)
    imports.to_csv(outfolder + 'importances.csv')
    
    # Out of sample predictions
    oos_preds = cross_val_predict(model.best_estimator_, x, y, cv=outer_strat)
    oos_results = pd.DataFrame([list(df['phone_number']), list(y), oos_preds]).T
    oos_results.columns = ['phone_number', 'true', 'predicted']
    oos_results.to_csv(outfolder + 'oos_predictions.csv', index=False)

In [ ]:
# Run machine learning model for each outcome of interest
for outcome in ['consumption', 'pmt', 'assetindex']:
    ml('data/features2018.csv', 'data/survey2018.csv', outcome, 'outputs/ml/' + outcome + '/')

In [ ]:
# Use "old" machine learning model to predict on "new" data
old_model = load('outputs/ml/pmt2018/LGBM/model')
new_data = pd.read_csv('data/features2018.csv')
predictions = old_model.predict(new_data.drop('phone_number', axis=1))
predictions = pd.DataFrame([list(new_data['phone_number']), predictions]).T
predictions.columns = ['phone_number', 'prediction']
predictions.to_csv('outputs/ml/consumption/temporality/old_model_new_data.csv')

### Table S12

In [ ]:
table = []
outcomes = ['consumption', 'pmt', 'assetindex']
outcome_names = ['Consumption', 'PMT', 'Asset Index']

for outcome in outcomes:
    results = pd.read_csv('outputs/ml/' + outcome + '/LGBM/oos_predictions.csv')
    singlefeature = pd.read_csv('data/single_feature2018.csv')
    results = results.merge(singlefeature, on='phone_number', how='inner')
    table.append([np.corrcoef(results['true'], results['predicted'])[0][1],
                  np.corrcoef(results['true'], results['single_feature'])[0][1]])
    
table = pd.DataFrame(table).T
table.columns = outcome_names
table = table.round(2)
table

### Table S4

In [ ]:
outcome = 'consumption'
importances = pd.read_csv('outputs/ml/' + outcome + '/LGBM/importances.csv')\
    [['Feature', 'Importance']]\
    .sort_values('Importance', ascending=False)
importances = importances[:10]
importances

### Figure S10

In [ ]:
sns.set(font_scale=2, style='white')
outcome = 'consumption'
poverty_line = 7.5
display_features = ['feature0', 'feature1', 'feature2', 'feature3']

results = pd.read_csv('outputs/ml/' + outcome + '/LGBM/oos_predictions.csv')
features = pd.read_csv('data/features2018.csv')[['phone_number'] + display_features]
results = results.merge(features, on='phone_number', how='inner')

fig, ax = plt.subplots(2, 2, figsize=(20, 8))
ax = ax.flatten()
for f, feat in enumerate(display_features):
    sns.kdeplot(results[results['true'] < poverty_line][feat], ax=ax[f], shade=True, label='Poor')
    sns.kdeplot(results[results['true'] >= poverty_line][feat], ax=ax[f], shade=True, label='Nonpoor')
    ax[f].set_title(feat.title())
    simpleaxis(ax[f])
ax[0].set_ylabel('Density')
ax[2].set_ylabel('Density')
plt.tight_layout()
plt.show()

### Figure S11

In [ ]:
outcome = 'consumption'
results = pd.read_csv('outputs/ml/' + outcome + '/LGBM/oos_predictions.csv')
homes = pd.read_csv('data/inferred_home_locations2018.csv')
results = results.merge(homes, on='phone_number', how='inner')

prefectures = gpd.read_file('data/shapefiles/prefectures.geojson')
cantons = gpd.read_file('data/shapefiles/cantons.geojson')

by_prefecture = results.groupby('prefecture', as_index=False).agg('mean')[['prefecture', 'predicted']]
counts_by_prefecture = results.groupby('prefecture', as_index=False).agg('count')[['prefecture', 'predicted']]\
    .rename({'predicted':'count'}, axis=1)
by_prefecture = by_prefecture.merge(counts_by_prefecture, on='prefecture', how='inner')
by_prefecture = prefectures.merge(by_prefecture, on='prefecture', how='inner')

by_canton = results.groupby('canton', as_index=False).agg('mean')[['canton', 'predicted']]
counts_by_canton = results.groupby('canton', as_index=False).agg('count')[['canton', 'predicted']]\
    .rename({'predicted':'count'}, axis=1)
by_canton = by_canton.merge(counts_by_canton, on='canton', how='inner')
by_canton = cantons.merge(by_canton, on='canton', how='inner')

In [ ]:
# Maps
sns.reset_orig()
fig, ax = plt.subplots(1, 2, figsize=(10, 10))

prefectures.plot(ax=ax[0], color='lightgrey')
by_prefecture.plot(ax=ax[0], column='predicted', legend=True, legend_kwds={'shrink':0.5})

cantons.plot(ax=ax[1], color='lightgrey')
by_canton.plot(ax=ax[1], column='predicted', legend=True, legend_kwds={'shrink':0.5})

ax[0].axis('off')
ax[1].axis('off')

plt.show()

In [ ]:
# Validation against survey data
sns.set(font_scale=1.5, style='white')
survey = pd.read_csv('data/survey2018.csv')
survey_by_prefecture = survey.groupby('prefecture', as_index=False).agg('mean')[['prefecture', outcome]]
survey_by_canton = survey.groupby('canton', as_index=False).agg('mean')[['canton', outcome]]

survey_by_prefecture = by_prefecture.merge(survey_by_prefecture, on='prefecture', how='inner')
survey_by_prefecture['consumption'] = np.log(survey_by_prefecture['consumption'])
survey_by_canton = by_canton.merge(survey_by_canton, on='canton', how='inner')
survey_by_canton['consumption'] = np.log(survey_by_canton['consumption'])

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 6))

ax[0].scatter(survey_by_prefecture[outcome], survey_by_prefecture['predicted'], 
              s=survey_by_prefecture['count']*150, alpha=.5, c='darkorange', edgecolor='black')
ax[1].scatter(survey_by_canton[outcome], survey_by_canton['predicted'], 
              s=survey_by_canton['count']*200, alpha=.5, c='dodgerblue', edgecolor='black')

p = np.corrcoef(survey_by_prefecture[outcome], survey_by_prefecture['predicted'])[0][1]
s = spearmanr(survey_by_prefecture[outcome], survey_by_prefecture['predicted'])[0]
ax[0].annotate('Pearson: %.2f\nSpearman: %.2f' % (p, s), xy=[.6, .2], xycoords='axes fraction', 
                 bbox=dict(facecolor='none', edgecolor='black', boxstyle='round,pad=1'))

p = np.corrcoef(survey_by_canton[outcome], survey_by_canton['predicted'])[0][1]
s = spearmanr(survey_by_canton[outcome], survey_by_canton['predicted'])[0]
ax[1].annotate('Pearson: %.2f\nSpearman: %.2f' % (p, s), xy=[.1, .8], xycoords='axes fraction', 
                 bbox=dict(facecolor='none', edgecolor='black', boxstyle='round,pad=1'))

simpleaxis(ax[0])
simpleaxis(ax[1])

for a in range(len(ax)):
    ax[a].set_title('Validation Against Survey Data')
    ax[a].set_xlabel('Average Consumption from Survey')
    ax[a].set_ylabel('Average Phone-Inferred Consumption')
plt.show()

### Table S19

In [ ]:
lgbm_cv = Pipeline([('winsorizer', Winsorizer()), 
                     ('model', LGBMRegressor(n_jobs=-1, random_state=2))])

In [ ]:
def classifier(feats_fname, survey_fname, outcome, toplevel_dir, rural_restriction=False, grid=None):
    
    # Load features
    feats = pd.read_csv(feats_fname)
    print('CDR observations: %i (%i unique)' % (len(feats), len(feats['phone_number'].unique())))
    
    # Load labels
    survey = pd.read_csv(survey_fname)
    if rural_restriction:
        survey = survey[survey['milieu'] == 'rural']
    if outcome in ['pmt', 'consumption', 'cons']:
        survey[outcome] = np.log(survey[outcome])
    print('Survey observations: %i (%i unique)' % (len(survey), len(survey['phone_number'].unique())))
    
    # Merge features and labels
    survey = survey.dropna(subset=[outcome])
    print('Survey observations with outcome: %i (%i unique)' % 
          (len(survey), len(survey['phone_number'].unique())))
    df = survey[['phone_number', outcome, 'weight']].merge(feats, on='phone_number')
    print('Final number of matched observations: %i (%i unique)' % (len(df), len(df['phone_number'].unique())))
    
    # Get X and Y 
    x = df.drop(['phone_number', 'weight', outcome], axis=1)
    y = df[outcome]   
    
    modelname, model, grid = 'LGBM', lgbm_cv, lgbm_grid
    print('Running tuned ' + modelname + '....')
    
    # Make folder structure
    if not os.path.isdir(toplevel_dir):
        os.mkdir(toplevel_dir)
    outfolder = toplevel_dir + modelname + '/'
    if os.path.isdir(outfolder):
        shutil.rmtree(outfolder)
    os.mkdir(outfolder)

    # Train model over all data, tuning hyperparameters with 3 fold CV
    model = GridSearchCV(estimator=model,
                         param_grid=grid,
                         cv=inner_strat,
                         verbose=0,
                         scoring='roc_auc',
                         return_train_score=True,
                         refit=True,
                         n_jobs=-1)
    model.fit(x, y)

    # Write the model itself
    dump(model, outfolder + 'model')

    # Record tuning results
    resultsdf = pd.DataFrame(model.cv_results_)
    resultsdf.to_csv(outfolder + 'tuning.csv', index=False)
    best_row = resultsdf.iloc[resultsdf['mean_test_score'].argmax()]
    print('Best r2 --- Train: %.2f (%.2f), Test: %.2f (%.2f)' % (best_row['mean_train_score'], 
                                                                 best_row['std_train_score'],
                                                                 best_row['mean_test_score'],
                                                                 best_row['std_test_score']))
    print('Best Hyperparameters: ' + json.dumps(resultsdf.iloc[resultsdf['mean_test_score'].argmax()]['params']))

    # Feature importances
    try:
        imports = model.best_estimator_.named_steps['model'].feature_importances_
    except:
        imports = model.best_estimator_.named_steps['model'].coef_
    imports = pd.DataFrame([x.columns, imports]).T
    imports.columns = ['Feature', 'Importance']
    imports = imports.sort_values('Importance', ascending=False)
    imports.to_csv(outfolder + 'importances.csv')
    
    # Out of sample predictions
    oos_preds = cross_val_predict(model.best_estimator_, x, y, cv=outer_strat)
    oos_results = pd.DataFrame([list(df['phone_number']), list(y), oos_preds]).T
    oos_results.columns = ['phone_number', 'true', 'predicted']
    oos_results.to_csv(outfolder + 'oos_predictions.csv', index=False)

In [ ]:
# Run machine learning model for each outcome of interest
for outcome in ['responded']:
    classifier('data/features2020.csv', 'data/survey2020.csv', outcome, 'outputs/ml/' + outcome + '/')

In [ ]:
outcome = 'responded'
importances = pd.read_csv('outputs/ml/' + outcome + '2020/LGBM/importances.csv')\
    [['Feature', 'Importance']]\
    .sort_values('Importance', ascending=False)
importances = importances[:10]
importances